In [6]:
import torch
import numpy as np
import pandas as pd
import os

# ==========================================
# 1. Configuration (Path Settings)
# ==========================================

# Input file path (As specified by you)
input_file_path = '/Users/oushilin/Desktop/Infor 301/final/Analysis_Ready_Data.pt'

# Output Excel file path (Saved in the same directory)
output_excel_path = '/Users/oushilin/Desktop/Infor 301/final/Yearly_City_Differences.xlsx'

print(f"Loading data from: {input_file_path}")

# ==========================================
# 2. Data Loading & Preparation
# ==========================================

try:
    # Load the .pt file
    data = torch.load(input_file_path)
except FileNotFoundError:
    print(f"Error: The file '{input_file_path}' was not found.")
    print("Please check the path and try again.")
    exit()
except Exception as e:
    print(f"An error occurred while loading the file: {e}")
    exit()

# Extract trajectories and city mapping
# trajectories structure: List of 10 arrays, each shape (10, 64) -> (Time, Features)
trajectories = data['trajectories']
city_map = data['city_map']

# Create a mapping from ID to City Name to ensure correct labeling
id_to_city = {v: k for k, v in city_map.items()}
num_cities = len(trajectories)
# Sort cities by ID to match the list index in 'trajectories'
cities = [id_to_city[i] for i in range(num_cities)]

print(f"Successfully loaded data for {num_cities} cities.")

# ==========================================
# 3. Calculation & Saving to Excel
# ==========================================

# Determine the number of years (time steps)
# Assuming shape is (Time, Features), so shape[0] is time
num_years = trajectories[0].shape[0]

print(f"Processing {num_years} years of data...")

# Use pandas ExcelWriter to save multiple sheets into one file
try:
    with pd.ExcelWriter(output_excel_path, engine='openpyxl') as writer:
        for t in range(num_years):
            # Initialize a zero matrix for the current year
            dist_matrix_t = np.zeros((num_cities, num_cities))
            
            # Calculate pairwise Euclidean distance for Year t
            for i in range(num_cities):
                for j in range(num_cities):
                    # Extract feature vector for City i at Year t
                    vec_i = trajectories[i][t, :]
                    # Extract feature vector for City j at Year t
                    vec_j = trajectories[j][t, :]
                    
                    # Compute Euclidean distance (L2 norm)
                    dist = np.linalg.norm(vec_i - vec_j)
                    dist_matrix_t[i, j] = dist
            
            # Create a DataFrame for the current year
            df_year = pd.DataFrame(dist_matrix_t, index=cities, columns=cities)
            
            # Write this DataFrame to a specific sheet in the Excel file
            sheet_name = f'Year_{t + 1}'
            df_year.to_excel(writer, sheet_name=sheet_name)
            
            print(f" - Processed {sheet_name}")

    print("\n" + "="*50)
    print("SUCCESS!")
    print(f"The file has been saved to:\n{output_excel_path}")
    print("="*50)

except Exception as e:
    print(f"\nError while saving Excel file: {e}")
    print("Make sure the file is not currently open in Excel.")

Loading data from: /Users/oushilin/Desktop/Infor 301/final/Analysis_Ready_Data.pt
Successfully loaded data for 10 cities.
Processing 10 years of data...
 - Processed Year_1
 - Processed Year_2
 - Processed Year_3
 - Processed Year_4
 - Processed Year_5
 - Processed Year_6
 - Processed Year_7
 - Processed Year_8
 - Processed Year_9
 - Processed Year_10

SUCCESS!
The file has been saved to:
/Users/oushilin/Desktop/Infor 301/final/Yearly_City_Differences.xlsx
